In [1]:
%matplotlib inline
import sys, os, time

import numpy as np
import matplotlib.pyplot as plt

from keras import backend as K
from keras.utils import to_categorical
from keras.applications.vgg19 import VGG19, preprocess_input

Using TensorFlow backend.


## Load data

In [2]:
x_val = np.load("/mnt/dados/imagenet/ILSVRC2012_img_val_224/x_val.npy") # loaded as RGB
x_val = preprocess_input(x_val) # converted to BGR

In [3]:
y_val = np.load("/mnt/dados/imagenet/ILSVRC2012_img_val_224/y_val.npy")

In [4]:
y_val_one_hot = to_categorical(y_val, 1000)

In [5]:
keras_idx_to_name = {}
f = open("/mnt/dados/imagenet/ILSVRC2012_img_val_224/synset_words.txt","r")
idx = 0
for line in f:
    parts = line.split(" ")
    keras_idx_to_name[idx] = " ".join(parts[1:])
    idx += 1
f.close()

## Benchmark models

In [6]:
def top_k_accuracy(y_true, y_pred, k=1):
    '''From: https://github.com/chainer/chainer/issues/606
    
    Expects both y_true and y_pred to be one-hot encoded.
    '''
    argsorted_y = np.argsort(y_pred)[:,-k:]
    return np.any(argsorted_y.T == y_true.argmax(axis=1), axis=0).mean()

In [7]:
K.clear_session()
model = VGG19()

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
y_pred = model.predict(x_val[:1000], verbose=1)

1000/1000 [==============================] - 9s 9ms/step


#### Top-1 Accuracy

Compare to 0.713 from Keras documentation

In [12]:
top_k_accuracy(y_val_one_hot[:1000], y_pred, k=1)

0.702

#### Top-5 Accuracy

Compare to 0.900 from Keras documentation

In [14]:
top_k_accuracy(y_val_one_hot[:1000], y_pred, k=5)

0.904